<a href="https://colab.research.google.com/github/nicolaiberk/Imbalanced/blob/master/02b_SMOTE_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Synthetic Minority Oversampling TEchnique - SMOTE

**The solution for this notebook can be found [here](https://colab.research.google.com/github/nicolaiberk/Imbalanced/blob/master/02b_SMOTE_Solution.ipynb).**

One way to address imbalanced data problems is to generate additional artificial cases. One method to do this is called **S**ynthetic **M**inority **O**versampling **Te**chnique - or short: **SMOTE**. The full paper can be found [here](https://arxiv.org/abs/1106.1813), a less technical explainer [here](https://www.analyticsvidhya.com/blog/2020/10/overcoming-class-imbalance-using-smote-techniques/) and the scikit-learn documentation on oversampling [here](https://imbalanced-learn.org/stable/over_sampling.html).

We use a highly imbalanced dataset of tweets that were annotated using crowd-coding from [this paper](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3026393). The tweets are classified according to whether they discuss the refugee movements in 2015. We will use the `imblearn` package to implement SMOTE.

We start by importing the packages and loading the data. 

In [ ]:
# oversample training samples for crossvalidation
import pandas as pd

# load relevant packages
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

# import SMOTE package
from imblearn.over_sampling import SMOTE

tweets = pd.read_csv("https://www.dropbox.com/s/gv56nu1ptrp63ps/annotated_german_refugee_tweets.csv?dl=1")

Divide into training and test set.

In [ ]:
# divide into training and test set
X_train, X_test, y_train, y_test = train_test_split(
    ___,
    ___,
    test_size=___,
    random_state=42
)
X_train = np.array(X_train)
X_test = np.array(X_test)

How imbalanced is the data?

In [ ]:
print(f"{sum(y_train)} positive cases out of a full set of {len(y_train)} in the training set ({round(sum(y_train)*100/len(y_train), 2)}%).")
pd.Series(y_train).value_counts().plot(kind='bar', title='Class distribution in raw training set')

Only 3% of the training data belong to the class of interest! Can we fix this by applying SMOTE?

# How does SMOTE work?

Doesn't look great, huh? We could just undersample, meaning we drop cases from the majority class (in this cases tweets *not* about refugees) until we have a balanced data set. However, we would end up with only a fraction of the observations and would lose a lot of information.

SMOTE instead **over**samples the minority class "by taking each minority class sample and introducing synthetic examples along the line segments joining any/all of the k minority class nearest neighbors" (p.328). 

In English: We take one observation in the minority class (in this case tweets) and look for similar minority class observations using a *k-nearest-neighbors* approach (more on this approach [here](https://medium.com/towards-data-science/machine-learning-basics-with-the-k-nearest-neighbors-algorithm-6a6e71d01761)). For a random subset of these similar neighbors, we generate a synthetic observation somewhere in the middle between the original observation and the chosen neighbor (see visualisation below, shamelessly stolen from the scikit-learn documentation).

![](https://imbalanced-learn.org/stable/_images/sphx_glr_plot_illustration_generation_sample_001.png)

# Application

Enough theory, let's just see how this works! First, we need to transform our texts into numbers by translating them into a document-term-matrix. For simplicity, we use the simple count-based matrix here, but you might use tfidf or any other method that produces some number indicating the frequency of a word in a document as well.

In [ ]:
# define Vectorizer
vec = TfidfVectorizer() # usually we would add some arguments here
X_train_dtm = vec.fit_transform(X_train)

Now witness the beauty of simplicity:

In [ ]:
# resample with SMOTE ()
X_resample, y_resample = SMOTE(sampling_strategy = _).fit_resample(__, __)

In [ ]:
print(f"{sum(y_resample)} positive cases out of a full set of {len(y_resample)} in the training set ({sum(y_resample)/len(y_resample)*100}%).")
pd.Series(y_resample).value_counts().plot(kind='bar', title='Class distribution in raw training set')

![](https://media3.giphy.com/media/zIwIWQx12YNEI/giphy.gif?cid=ecf05e47yuqbrs13qjl921umf83eb1m4nhnpcstrc38zvcuk&rid=giphy.gif&ct=g)

# Performance

So now we turned our 3% minority class into 50%! (Note: more careful design of this ratio should lead to better results)

Let's see how it performs. First, we train a classifier on the unSMOTE'd, imbalanced data:

In [ ]:
from sklearn.linear_model import 

# fit
pipe = Pipeline([('tfidf', TfidfVectorizer()), ('', )])
pipe.___(X_train, y_train)

# classify test set and show performance
y_pred_m = pipe.___(X_test)

print("Accuracy: ", accuracy_score(y_test, y_pred_m).round(2))
print("Recall: ", recall_score(y_test, y_pred_m).round(2))
print("Precision: ", precision_score(y_test, y_pred_m).round(2))
print("F1: ", f1_score(y_test, y_pred_m).round(2))
pd.crosstab(y_test, y_pred_m)

Here we have very good precision, but would only find very few of the cases in the outcome category, which are the ones we care about after all. 

How 'bout SMOTE?

In [ ]:
# transform test set
X_test_dtm = vec.___(X_test) 
## note it's transform, not fit_transform, as we transform it into the same dtm as the training set

# define and fit classifier
clsfr = ___
clsfr.fit(____, _____)

# predict
y_pred = clsfr.predict(X_test_dtm)

print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Recall: ", round(recall_score(y_test, y_pred), 2))
print("Precision: ", round(precision_score(y_test, y_pred), 2))
print("F1: ", round(f1_score(y_test, y_pred), 2))
pd.crosstab(y_test, y_pred)

SMOTE for the win!

---

For more info on SMOTE and its variants, check out the [scikit-learn documentation on combining under- and oversampling](https://imbalanced-learn.org/stable/combine.html).